In [2]:
# Interval = min K

 # !python -m "pyalgotrade.tools.quandl" --source-code="WIKI" --table-code="aapl" --from-year=2000 --to-year=2000 --storage=. --force-download --frequency=daily
import yfinance as yf
from datetime import datetime
import pandas as pd

# ticker = '^gspc'
ticker = 'tsla'

start_date = datetime(2022, 9, 30,21)
end_date = datetime(2022, 10, 1,16)
data = yf.download(ticker, interval = '2m', start = start_date ,end = end_date, auto_adjust = False)
data.index.name = 'Date Time'
data.index = pd.to_datetime(data.index, format = '%m/%d/%Y').strftime('%Y-%m-%d %H:%M:%S')
data.to_csv('data.csv')
print(data)

# start_date = datetime(2022, 9, 26)
# end_date = datetime(2022, 9, 27)
interval1 = '1m'
interval2 = '2m'
data1 = yf.download(ticker, interval = interval1, start = start_date ,end = end_date, auto_adjust = False)
data1.index.name = 'Date Time'
data1.index = pd.to_datetime(data1.index, format = '%m/%d/%Y').strftime('%Y-%m-%d %H:%M:%S')
data1.to_csv('data' + interval1 + '.csv')
print(data1)

data2 = yf.download(ticker, interval = interval2, start = start_date ,end = end_date, auto_adjust = False)
data2.index.name = 'Date Time'
data2.index = pd.to_datetime(data2.index, format = '%m/%d/%Y').strftime('%Y-%m-%d %H:%M:%S')
data2.to_csv('data' + interval2 + '.csv')
print(data2)

[*********************100%***********************]  1 of 1 completed
                           Open        High         Low       Close  \
Date Time                                                             
2022-09-30 09:30:00  266.144989  266.191986  263.630005  265.500000   
2022-09-30 09:32:00  265.440796  267.799988  264.890991  267.725006   
2022-09-30 09:34:00  267.790009  268.859985  267.649506  268.160004   
2022-09-30 09:36:00  268.000000  268.070099  263.329987  263.433014   
2022-09-30 09:38:00  262.750000  264.570007  262.470001  264.570007   
...                         ...         ...         ...         ...   
2022-09-30 15:52:00  266.170013  266.239990  265.589996  265.859985   
2022-09-30 15:54:00  265.813995  266.000000  265.570007  265.600006   
2022-09-30 15:56:00  265.600006  265.640015  265.130005  265.299988   
2022-09-30 15:58:00  265.286285  265.649994  265.000000  265.200012   
2022-09-30 16:00:00  265.250000  265.250000  265.250000  265.250000   

       

In [4]:
# Interval = 2 min K

# from __future__ import print_function

from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.technical import ma
from pyalgotrade.technical import rsi
from pyalgotrade.technical import stoch
from pyalgotrade.bar import Frequency


number_of_shares = 10
kdj = []
kdjd = []
rsi1 = []

def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 100000) # initial equity
        
        # We'll use adjusted close values instead of regular close values.
        # self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), smaPeriod)
        self.__rsi = rsi.RSI(feed[instrument].getCloseDataSeries(), 6)
        self.__kdj = stoch.StochasticOscillator(feed[instrument], period=9, dSMAPeriod=3)
        self.D = self.__kdj.getD()
        self.__position = None
        self.__instrument = instrument

    def onEnterOk(self, position):
        global execInfo
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f %s %s %s" % (execInfo.getPrice(),safe_round(self.__kdj[-2], 2),safe_round(self.D[-2], 2),self.__position.getShares()))
        global buy, total_buy
        buy = execInfo.getPrice() * number_of_shares
        total_buy += buy

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f %s %s %s" % (execInfo.getPrice(),safe_round(self.__kdj[-2], 2),safe_round(self.D[-2], 2),self.__position.getShares()))
        # self.__position = None
        sell = execInfo.getPrice() * number_of_shares
        global return_per_share, total_sell
        total_sell += sell
        return_per_share = total_sell - total_buy

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return
        
        kdj.append(self.__kdj[-1])
        kdjd.append(self.D[-1])
        rsi1.append(self.__rsi[-1])
        # for i in enumerate(self.__rsi):
        #     print(i)
        # ALL 30 70 (20)(14)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLongStop(self.__instrument, stopPrice=execInfo.getPrice() * 0.995, quantity=number_of_shares, goodTillCanceled=True)
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShortStop(self.__instrument, stopPrice=execInfo.getPrice() * 1.005, quantity=number_of_shares, goodTillCanceled=True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShortStop(self.__instrument, stopPrice=execInfo.getPrice() * 1.005, quantity=number_of_shares, goodTillCanceled=True)
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLongStop(self.__instrument, stopPrice=execInfo.getPrice() * 0.995, quantity=number_of_shares, goodTillCanceled=True)


        # ALL 30 70 (13)(15)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # ALL 20 80 (8.6)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # ALL 20 80 
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # Just < 30 and > 70 (17)(15)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1], 2) < 30 and safe_round(self.__rsi[-1], 2) < 30:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) > 70 and safe_round(self.__rsi[-1], 2) > 70:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1], 2) > 70 and safe_round(self.__rsi[-1], 2) > 70 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) < 30 and safe_round(self.__rsi[-1], 2) < 30 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # Just < 20 and > 80 (10)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1], 2) < 20 and safe_round(self.__rsi[-1], 2) < 20:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) > 80 and safe_round(self.__rsi[-1], 2) > 80:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1], 2) > 80 and safe_round(self.__rsi[-1], 2) > 80 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) < 20 and safe_round(self.__rsi[-1], 2) < 20 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # KDJ cross (15.4)
        # if self.__position is None:
        #     # print(self.__kdj[-1],self.D[-1])
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        if self.__position is None:
            if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2):
                self.__position = self.enterLong(self.__instrument, number_of_shares, True)
            if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2):
                self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        else:
            if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
                self.__position.exitMarket()
                self.__position = self.enterShort(self.__instrument, number_of_shares, True)
            if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
                self.__position.exitMarket()
                self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # trial 2 (9)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) < 50 and safe_round(self.__rsi[-2], 2) < 50:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) > 50 and safe_round(self.__rsi[-2], 2) > 50:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # trial 3 (17)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) < 50 and safe_round(self.__rsi[-2], 2) < 50:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) > 50 and safe_round(self.__rsi[-2], 2) > 50:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)


def run_strategy(smaPeriod):
    global total_sell, total_buy
    total_buy = 0
    total_sell = 0
    # Load the bar feed from the CSV file
    # feed = quandlfeed.Feed()
    feed = csvfeed.GenericBarFeed(Frequency.MINUTE)
    feed.addBarsFromCSV(ticker,'data.csv')

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, ticker, smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())
    # print('Return = ', return_per_share)

run_strategy(15)


2022-09-30 10:00:00 strategy [INFO] BUY at $268.96 64.49 83.16 -10
2022-09-30 10:04:00 strategy [INFO] SELL at $269.28 55.83 54.9 0
2022-09-30 10:04:00 strategy [INFO] BUY at $269.28 55.83 54.9 10
2022-09-30 10:08:00 strategy [INFO] SELL at $268.63 27.15 47.0 0
2022-09-30 10:08:00 strategy [INFO] BUY at $268.63 27.15 47.0 -10
2022-09-30 10:10:00 strategy [INFO] SELL at $269.92 58.54 47.9 0
2022-09-30 10:10:00 strategy [INFO] BUY at $269.92 58.54 47.9 10
2022-09-30 10:14:00 strategy [INFO] SELL at $270.07 60.29 69.73 0
2022-09-30 10:14:00 strategy [INFO] BUY at $270.07 60.29 69.73 -10
2022-09-30 10:20:00 strategy [INFO] SELL at $270.22 64.97 54.7 0
2022-09-30 10:20:00 strategy [INFO] BUY at $270.22 64.97 54.7 10
2022-09-30 10:22:00 strategy [INFO] SELL at $269.65 40.94 49.16 0
2022-09-30 10:22:00 strategy [INFO] BUY at $269.65 40.94 49.16 -10
2022-09-30 10:24:00 strategy [INFO] SELL at $270.20 56.82 54.24 0
2022-09-30 10:24:00 strategy [INFO] BUY at $270.20 56.82 54.24 10
2022-09-30 10:

Final portfolio value: $99945.13


In [502]:
import plotly.graph_objs as go

print(len(kdj),len(kdjd),len(rsi1))

trace1 = go.Scatter(
    x=data.index,
    y=kdj,
    mode='lines',
    name='KDJ K Line'
)

trace2 = go.Scatter(
    x=data.index,
    y=kdjd,
    mode='lines',
    name='KDJ D Line'
)

trace3 = go.Scatter(
    x=data.index,
    y=rsi1,
    mode='lines',
    name='RSI1 Line'
)

layout = go.Layout(
    title='indicators',
    xaxis={'title': "Time"},
    yaxis={'title': "kdj/rsi"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

182 182 182


In [13]:
from more_itertools import collapse
from pyalgotrade.barfeed import quandlfeed

ticker = 'spy'

import nasdaqdatalink

NASDAQ_DATA_LINK_API_KEY = 'Mx8UZkQJfsJiRneDiPaK'

nasdaqdatalink.ApiConfig.api_key = (NASDAQ_DATA_LINK_API_KEY)

data = nasdaqdatalink.get('WIKI/TSLA')

print(data)

              Open      High     Low   Close      Volume  Ex-Dividend  \
Date                                                                    
2010-06-29   19.00   25.0000   17.54   23.89  18766300.0          0.0   
2010-06-30   25.79   30.4192   23.30   23.83  17187100.0          0.0   
2010-07-01   25.00   25.9200   20.27   21.96   8218800.0          0.0   
2010-07-02   23.00   23.1000   18.71   19.20   5139800.0          0.0   
2010-07-06   20.00   20.0000   15.83   16.11   6866900.0          0.0   
...            ...       ...     ...     ...         ...          ...   
2018-03-21  310.25  322.4400  310.19  316.53   5927881.0          0.0   
2018-03-22  313.89  318.8200  308.18  309.10   4914307.0          0.0   
2018-03-23  311.25  311.6100  300.45  301.54   6600538.0          0.0   
2018-03-26  307.34  307.5900  291.36  304.18   8324639.0          0.0   
2018-03-27  304.00  304.2700  277.18  279.18  13696168.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj

In [14]:
import quandl

quandl.ApiConfig.api_key = 'Mx8UZkQJfsJiRneDiPaK'

data = quandl.get('WIKI/TSLA')

# data = quandl.get('WIKI/TSLA/, ')

print(data)

              Open      High     Low   Close      Volume  Ex-Dividend  \
Date                                                                    
2010-06-29   19.00   25.0000   17.54   23.89  18766300.0          0.0   
2010-06-30   25.79   30.4192   23.30   23.83  17187100.0          0.0   
2010-07-01   25.00   25.9200   20.27   21.96   8218800.0          0.0   
2010-07-02   23.00   23.1000   18.71   19.20   5139800.0          0.0   
2010-07-06   20.00   20.0000   15.83   16.11   6866900.0          0.0   
...            ...       ...     ...     ...         ...          ...   
2018-03-21  310.25  322.4400  310.19  316.53   5927881.0          0.0   
2018-03-22  313.89  318.8200  308.18  309.10   4914307.0          0.0   
2018-03-23  311.25  311.6100  300.45  301.54   6600538.0          0.0   
2018-03-26  307.34  307.5900  291.36  304.18   8324639.0          0.0   
2018-03-27  304.00  304.2700  277.18  279.18  13696168.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj

In [ ]:
data = googlefeed.Feed()